<a href="https://colab.research.google.com/github/orlandoalexander/Moda-Personal-Assistant/blob/models/Nigel_Attribute_ModelTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import Input
import pandas as pd
import numpy as np


In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# from tensorflow import keras
# model = keras.models.load_model('/content/drive/MyDrive/Moda/saved_models/mobilenet_design_v1')

In [4]:
# model.call()

In [5]:
# !pip install --upgrade pip


In [6]:
!pip install git+https://github.com/orlandoalexander/Moda-Personal-Assistant.git@preproc_package


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/orlandoalexander/Moda-Personal-Assistant.git (to revision preproc_package) to /tmp/pip-req-build-88sf5qk0
  Running command git clone -q https://github.com/orlandoalexander/Moda-Personal-Assistant.git /tmp/pip-req-build-88sf5qk0
  Running command git checkout -b preproc_package --track origin/preproc_package
  Switched to a new branch 'preproc_package'
  Branch 'preproc_package' set up to track remote branch 'preproc_package' from 'origin'.
  Using cached matplotlib-3.5.2-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.3 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.6.2
    Uninstalling matplotlib-3.6.2:
      Successfully uninstalled matplotlib-3.6.2


In [7]:
# Uncomment as needed
# !unzip '/content/drive/MyDrive/attribute_img/img.zip' # Attribute
# !unzip '/content/drive/MyDrive/Moda/images.zip'

Archive:  /content/drive/MyDrive/Moda/images.zip
replace images/MEN-Denim-id_00000080-01_7_additional.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
N


In [8]:
from preproc.preprocess import SectionPreproc


In [9]:
# input size for resnet, efficientnet and mobilenet is (224,224). For inceptionv3 use 299,299
prep = SectionPreproc('images', (224,224), 0.2)

In [ ]:
X_train, X_test, y_train, y_test, names  = prep.run()

Augmenting section 'upper'...


In [ ]:
names

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(X_test[5000])

In [ ]:
np.save('X_train.npy',X_train)
np.save('X_test.npy',X_test)
np.save('y_train.npy',y_train)
np.save('y_test.npy',y_test)


In [ ]:
from matplotlib.pyplot import imshow

In [ ]:
names

In [ ]:
imshow(X_test[8])
y_test[8]

In [ ]:
import time

In [ ]:
# imshow(X_train[])
# y_train[]

In [ ]:
pd.DataFrame(y_train).value_counts()

In [ ]:
## New Class code
y_train[:, :4]

In [ ]:
## Orginal Class code

from keras.layers import Input, Dense, Flatten, GlobalAveragePooling2D, Lambda, BatchNormalization, Dropout
from keras import Sequential, Model
from keras.metrics import Precision, Recall
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

from keras.applications import InceptionV3
from keras.applications import inception_v3
from keras.applications import ResNet50
from keras.applications import MobileNetV2
from keras.applications import MobileNet
from keras.applications import mobilenet_v2
from keras.applications import EfficientNetB0
from keras.applications import efficientnet
from keras.applications import mobilenet
from keras.applications import resnet
from keras.applications import inception_v3

from sklearn.model_selection import train_test_split

# attribute should be one of the following strings:
# 'design', 'sleeves', 'length', 'neckline', 'fabric', 'fit'

# model should be one of the following strings:
# 'inception', 'resnet', 'mobilenet', 'efficientnet'

class AttrModel(Model):
    def __init__(self, attribute, epochs, model_name, in_shape, batch_size, callbacks, final_layer_neurons,
                 pretrain_lr, finetune_lr,
                 X_train, X_test,
                 y_train, y_test, **kwargs):
        super().__init__()
        self.attribute = attribute
        self.model_name = model_name.lower()
        self.in_shape = in_shape
        self.batch_size = batch_size
        self.epochs = epochs
        self.pretrain_lr = pretrain_lr
        self.finetune_lr = finetune_lr
        self.callbacks = callbacks
        self.final_layer_neurons = final_layer_neurons
        self.kwargs = kwargs
        self.X_pretrain, self.X_finetune, self.y_pretrain, self.y_finetune = train_test_split(
            X_train, y_train, test_size=0.5
        )
        self.X_test = X_test
        self.y_test = y_test
        self.cat_nums = {
            'design': 7,
            'sleeves': 3,
            'length': 2,      # Binary category = 1
            'neckline': 4,
            'fabric': 6,
            'fit': 3,
            'section': 4
        }

        del X_train
        del y_train

        self.cat_num = self.cat_nums[self.attribute]
        self.activation = 'softmax'
        self.loss = 'categorical_crossentropy'
        self.model = self.instantiate_model()  # calling the function below

    def instantiate_model(self):
        input = Input(self.in_shape)
        if self.model_name == 'inception':              # calling the chosen pretrained model
            base_model = InceptionV3(include_top=False, weights='imagenet',
                                    classes=self.cat_num, input_shape=self.in_shape)
            self.preprocess_input = inception_v3.preprocess_input
        elif self.model_name == 'resnet':
            base_model = ResNet50(include_top=False, weights='imagenet',
                                  classes=self.cat_num, input_shape=self.in_shape)
            self.preprocess_input = resnet.preprocess_input
        elif self.model_name == 'mobilenet':
            base_model = MobileNetV2(include_top=False, weights='imagenet',
                                     classes=self.cat_num, input_shape=self.in_shape)
            self.preprocess_input = mobilenet.preprocess_input
        elif self.model_name == 'efficientnet':
            base_model = EfficientNetB0(include_top=False, weights='imagenet',
                                        classes=self.cat_num, input_shape=self.in_shape)
            self.preprocess_input = efficientnet.preprocess_input
        else:
            print('''No model found. Please pass one of the following:
                  inception, resnet, mobilenet, efficientnet''')

        base_model.trainable = False    # freeze layers
        outputs = base_model(input)
        outputs = BatchNormalization(name='BatchNormalization')(outputs)
        outputs = GlobalAveragePooling2D()(outputs)
        outputs = Dropout(0.5)(outputs)
        outputs = Dense(units=self.final_layer_neurons, activation='relu')(outputs)
        outputs = LeakyReLU(alpha=0.1)(outputs)
        outputs = Dropout(0.25)(outputs)
        outputs = Dense(units=self.cat_num, activation=self.activation)(outputs)
        self.model = Model(inputs = [input], outputs = [outputs])
        self.model.save('')
        print(self.loss) # check loss func
        self.model.compile(loss=self.loss, optimizer=Adam(learning_rate=self.pretrain_lr),
                      metrics=['accuracy', Precision(), Recall()])
        return self.model

    def train(self):
        self.datagen = ImageDataGenerator(preprocessing_function=self.preprocess_input)

        self.X_pretrain, self.X_val, self.y_pretrain, self.y_val = train_test_split(
            self.X_pretrain, self.y_pretrain, test_size=0.2
        )

        self.pretrain_generator = self.datagen.flow( 
        self.X_pretrain, self.y_pretrain,                                
        batch_size=16)

        self.pretrain_val = self.datagen.flow( 
        self.X_val, self.y_val,                                
        batch_size=16)

        # Train the model on the images and labels
        self.model.history = self.model.fit(self.pretrain_generator,validation_data = self.pretrain_val,
        epochs=self.epochs, batch_size=self.batch_size, verbose=1, 
        callbacks=self.callbacks)

        del self.pretrain_generator
        del self.pretrain_val

        return self.model.history

    def finetune(self,
                 metrics=['accuracy', Precision(), Recall()]):
        self.finetune_generator = self.datagen.flow( 
        self.X_finetune, self.y_finetune,                                
        batch_size=16)
      
        self.model.trainable = True     # unfreeze layers, then compile to save changes
        self.model.compile(
            optimizer=Adam(self.finetune_lr),                  # Very low learning rate
            loss=self.loss,
            metrics=metrics)
        self.model.fit(self.finetune_generator,
                       epochs=self.epochs, batch_size=self.batch_size,
                       callbacks=self.callbacks)
        
        del self.finetune_generator
        return self.model, self.model.history

    def evaluate(self):
        self.test_generator = self.datagen.flow( 
        self.X_test, self.y_test,                                
        batch_size=16)
        res = self.model.evaluate(self.test_generator, verbose=1)
        del self.test_generator
        return res
    def predict(self, X):
        return self.model.predict(self.preprocess_input(X))


    def save(self, filepath):
        return tf.keras.models.save_model(self.model,
                                   filepath=filepath)
        
    def summary(self):
        return self.model.summary()


In [ ]:
mobilenet_section = AttrModel(attribute='section',
                      model_name='mobilenet',
                      in_shape=(224,224,3),
                      batch_size= 32,
                      epochs=25,
                      pretrain_lr = .0001,
                      finetune_lr = .00001,
                      callbacks = EarlyStopping(patience=3, restore_best_weights=True, monitor='accuracy'),
                      final_layer_neurons=100,
                      X_train=X_train,
                      X_test=X_test,
                      y_train=y_train[:, :4],
                      y_test=y_test[:, :4])

In [ ]:
mobilenet_section.train()

In [ ]:
mobilenet_section.evaluate()

In [ ]:
mobilenet_section.finetune()

In [ ]:
mobilenet_section.summary()

In [ ]:
mobilenet_section.evaluate()

In [ ]:
mobilenet_length.save('/content/drive/MyDrive/Moda/saved_models/mobilenet_length_v1')

In [ ]:
mobilenet_length_v1_metrics = {
    'loss': 0.2144,
    'accuracy': 0.9276,
    'precision': 0.9276,
    'recall': 0.9276
    }

In [ ]:
import json

In [ ]:
# Writing to sample.json
with open('/content/drive/MyDrive/Moda/saved_models/mobilenet_length_v1/eval_metrics.txt', 'w') as outfile:
    outfile.write(json.dumps(mobilenet_length_v1_metrics))